In [14]:
import argparse
import logging
import os
import pprint
import threading
import time
import timeit
import traceback
import typing

os.environ["OMP_NUM_THREADS"] = "1"  # Necessary for multithreading.

import torch
from torch import multiprocessing as mp
from torch.multiprocessing import Process, Manager
from torch import nn
from torch.nn import functional as F

from torchbeast.core import file_writer
from torchbeast.core import prof
from torchbeast.core import vtrace
from torchbeast.core.environment import Environment, Vec_Environment

from torchbeast.atari_wrappers import *
from torchbeast.transformer_rnn import *
from torchbeast.train import *
from torchbeast.model import Model
from torchbeast.base import BaseNet

import gym
import gym_sokoban
import numpy as np
from matplotlib import pyplot as plt
import logging
from collections import deque

logging.basicConfig(format='%(message)s', level=logging.DEBUG)
logging.getLogger('matplotlib.font_manager').disabled = True

torch.multiprocessing.set_sharing_strategy('file_system')

def get_param(net, name=None):
    keys = []
    for (k, v) in actor_wrapper.model.named_parameters(): 
        if name is None:
            print(k)
        else:
            if name == k: return v
        keys.append(k)
    return keys

<font size="5">Agent Training Phase</font>

In [15]:
def gumbel_softmax(logits, temperature, u=None, hard=True):
    """
    ST-gumple-softmax
    input: [bsz, n_class]
    return: flatten --> [bsz, n_class] an one-hot vector
    """
    eps = 1e-20
    n_class = logits.shape[-1]
    
    if u is None:
        u = torch.rand(logits.size()).to(logits.device)
        u = -torch.log(-torch.log(u + eps) + eps)
    y = logits + u
    y = F.softmax(y / temperature, dim=-1)
    
    if not hard: return y.view(-1, n_class)

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard.view(-1, n_class), u

class Actor_net(nn.Module):    
    def __init__(self, obs_shape, num_actions, flags):

        super(Actor_net, self).__init__()
        self.obs_shape = obs_shape
        self.num_actions = num_actions  
        
        self.tran_t = flags.tran_t                   # number of recurrence of RNN
        self.tran_mem_n = flags.tran_mem_n           # size of memory for the attn modules
        self.tran_layer_n = flags.tran_layer_n       # number of layers
        self.tran_lstm = flags.tran_lstm             # to use lstm or not
        self.tran_lstm_no_attn = flags.tran_lstm_no_attn  # to use attention in lstm or not
        self.tran_norm_first = flags.tran_norm_first # to use norm first in transformer (not on LSTM)
        self.tran_ff_n = flags.tran_ff_n             # number of dim of ff in transformer (not on LSTM)        
        self.tran_skip = flags.tran_skip             # whether to add skip connection
        self.conv_out = flags.tran_dim               # size of transformer / LSTM embedding dim
        self.ste = flags.ste
        self.gb_ste = flags.gb_ste
        
        self.conv_out_hw = 1   
        self.d_model = self.conv_out
        
        self.conv1 = nn.Conv2d(in_channels=self.obs_shape[0], out_channels=self.conv_out//2, kernel_size=1, stride=1)        
        self.conv2 = nn.Conv2d(in_channels=self.conv_out//2, out_channels=self.conv_out, kernel_size=1, stride=1)        
        self.frame_conv = torch.nn.Sequential(self.conv1, nn.ReLU(), self.conv2, nn.ReLU())
        self.env_input_size = self.conv_out
        d_in = self.env_input_size + self.d_model 
        
        if self.tran_lstm:
            self.core = ConvAttnLSTM(h=self.conv_out_hw, w=self.conv_out_hw,
                                 input_dim=d_in-self.d_model, hidden_dim=self.d_model,
                                 kernel_size=1, num_layers=self.tran_layer_n,
                                 num_heads=8, mem_n=self.tran_mem_n, attn=not self.tran_lstm_no_attn)
        else:            
            self.core = ConvTransformerRNN(d_in=d_in,
                                       h=self.conv_out_hw, w=self.conv_out_hw, d_model=self.d_model, 
                                       num_heads=8, dim_feedforward=self.tran_ff_n, 
                                       mem_n=self.tran_mem_n, norm_first=self.tran_norm_first,
                                       num_layers=self.tran_layer_n, rpos=self.rpos, conv=False)   
                         
        
        if self.tran_skip:
            rnn_out_size = self.conv_out_hw * self.conv_out_hw * (self.d_model + self.env_input_size)
        else:
            rnn_out_size = self.conv_out_hw * self.conv_out_hw * self.d_model
                
        self.fc = nn.Linear(rnn_out_size, 256)        
        
        self.im_policy = nn.Linear(256, self.num_actions)        
        self.policy = nn.Linear(256, self.num_actions)        
        self.baseline = nn.Linear(256, 1)        
        self.reset = nn.Linear(256, 1)        
        
        if self.gb_ste:
            self.register_buffer('temp', torch.tensor(flags.gb_ste_temp_max, dtype=torch.float32))
        
        print("actor size: ", sum(p.numel() for p in self.parameters()))
        #for k, v in self.named_parameters(): print(k, v.numel())   

    def initial_state(self, batch_size):
        state = self.core.init_state(batch_size) + (torch.zeros(1, batch_size, 
               self.env_input_size, self.conv_out_hw, self.conv_out_hw),)
        return state

    def forward(self, x, done, core_state=(), u=None, debug=False):
        # one-step forward for the actor
        # input / done shape x: T x B x C x 1 x 1 / B x C x 1 x 1
        # only supports T = 1 at the moment; all output does not have T dim.
        
        if len(x.shape) == 4: x = x.unsqueeze(0)
        if len(done.shape) == 1: done = done.unsqueeze(0)  
            
        T, B, *_ = x.shape
        x = torch.flatten(x, 0, 1)  # Merge time and batch.        
        env_input = self.frame_conv(x)                
        core_input = env_input.view(T, B, -1, self.conv_out_hw, self.conv_out_hw)
        core_output_list = []
        notdone = ~(done.bool())
        
        for n, (input, nd) in enumerate(zip(core_input.unbind(), notdone.unbind())):                
            # Input shape: B, self.conv_out + self.num_actions + 1, H, W
            for t in range(self.tran_t):                
                if t > 0: nd = torch.ones(B).to(x.device).bool()                    
                nd = nd.view(-1)      
                output, core_state = self.core(input, core_state, nd, nd) # output shape: 1, B, core_output_size 
                
            last_input = input   
            core_output_list.append(output)
                                   
        core_output = torch.cat(core_output_list)  
        if self.tran_skip: core_output = torch.concat([core_output, core_input], dim=-3)
        core_output = torch.flatten(core_output, 0, 1)        
        core_output = F.relu(self.fc(torch.flatten(core_output, start_dim=1)))   
        
        policy_logits = self.policy(core_output)
        im_policy_logits = self.im_policy(core_output)
        baseline = self.baseline(core_output)
        reset_logits = self.reset(core_output)
        
        if self.gb_ste:
            if u is not None: 
                u_action = u[:, :-2]
                u_reset = u[:, -2:]
            else:
                u_action = None
                u_reset = None
            im_action, u_action = gumbel_softmax(im_policy_logits, self.temp, u=u_action, hard=True)
            n_reset_logits = torch.cat([reset_logits, torch.zeros_like(reset_logits)], dim=-1)    
            reset, u_reset = gumbel_softmax(n_reset_logits, self.temp, u=u_reset, hard=True)
            reset = reset[:, 0]
            u = torch.cat([u_action, u_reset], dim=-1)
        elif self.ste:
            im_action_p = F.softmax(im_policy_logits, dim=-1)
            im_action_h = torch.multinomial(im_action_p, num_samples=1)
            im_action = (im_action_h - im_action_p).detach() + im_action_p
          
            reset_p = torch.sigmoid(reset_logits)
            reset_h = torch.bernoulli(reset_p)
            reset = (reset_h - reset_p).detach() + reset_p
        else:
            im_action = F.softmax(im_policy_logits, dim=1)
            reset = torch.sigmoid(reset_logits)
            
        
        action = torch.multinomial(F.softmax(policy_logits, dim=1), num_samples=1)
        
        reg_loss = (1e-3 * torch.sum(policy_logits**2, dim=-1) / 2 + 
                    1e-5 * torch.sum(core_output**2, dim=-1) / 2)
        reg_loss = reg_loss.view(T, B)
        
        policy_logits = policy_logits.view(T, B, self.num_actions)
        action = action.view(T, B)                
        im_policy_logits = im_policy_logits.view(T, B, self.num_actions)
        im_action = im_action.view(T, B, self.num_actions)        
        baseline = baseline.view(T, B)
        reset = reset.view(T, B)     
        
        ret_dict = dict(policy_logits=policy_logits[0],                         
                        action=action[0], 
                        im_policy_logits=im_policy_logits[0],                         
                        im_action=im_action[0],                        
                        baseline=baseline[0], 
                        reset=reset[0],
                        reg_loss=reg_loss[0], )
        if self.gb_ste:
            ret_dict['uniform'] = u
        return (ret_dict, core_state)      
    
class Actor_Wrapper(nn.Module):    
    def __init__(self, flags, model, actor=None, rec_t=5):
        
        super(Actor_Wrapper, self).__init__()   
        self.model = model
        self.num_actions = model.num_actions        
        self.rec_t = rec_t
        self.obs_shape = (3 * num_actions + 4 + self.rec_t, 1, 1)        
        if actor is None:
            self.actor = Actor_net(obs_shape=self.obs_shape, num_actions=self.num_actions, flags=flags)
        else:
            self.actor = actor   
            
    def initial_state(self, batch_size):
        return self.actor.initial_state(batch_size)
            
    def forward(self, x, core_state=None):                      
        # x is env_output object with:
        # frame: T x B x C x H x W
        # last_action: T x B
        # reward: T x B
        
        tot_step, bsz, _, _, _ = x['frame'].shape
        device = x['frame'].device        
        self.model.train(False)
        
        for step in range(tot_step):
        
            state = x['frame'][step]        
            action = F.one_hot(x['last_action'][step], self.num_actions)
            reward = x['reward'][step]   
            done = x['done'][step]
            
            rs, vs, logits, encodeds = self.model(state, action.unsqueeze(0), one_hot=True)
            encoded_reset = encodeds[0].clone()
            encoded = encodeds[0]

            for t in range(self.rec_t): 
                
                a = action if t == 0 else im_action  
                r = (reward if t == 0 else rs[-1]).unsqueeze(-1)
                v = vs[-1].unsqueeze(-1)
                logit = logits[-1]                    
                
                if t == 0:
                    r0 = r.clone()
                    v0 = v.clone()
                    logit0 = logit.clone()
                    if self.actor.gb_ste: u_list = []
                
                time = F.one_hot(torch.tensor([t], device=device).long(), self.rec_t).tile([bsz, 1])                        

                actor_input = torch.concat([a, r, v, logit, r0, v0, logit0, time], dim=-1)     
                actor_input = actor_input.unsqueeze(-1).unsqueeze(-1)
                
                if 'uniform' in x.keys():
                    u = x['uniform'][step][:, t]
                else:
                    u = None                    
                
                actor_output, core_state = self.actor(actor_input, done=done, core_state=core_state,
                                                      u=u)                
                if self.actor.gb_ste: u_list.append(actor_output['uniform'].unsqueeze(1))
                
                if t == self.rec_t - 1:
                    if self.actor.gb_ste:
                        actor_output["uniform"] = torch.concat(u_list, dim=1)
                    if step == 0:
                        all_actor_output = {k: [v.unsqueeze(0)] for k, v in actor_output.items()}
                    else:
                        for k, v in actor_output.items(): all_actor_output[k].append(v.unsqueeze(0))
                
                reset = actor_output["reset"]
                im_action = actor_output["im_action"]

                if t < self.rec_t - 1:                
                    rs, vs, logits, encodeds = self.model.forward_encoded(encoded, im_action.unsqueeze(0), 
                                                                          one_hot=True)
                    reset = reset.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
                    encoded = reset * encoded_reset + (1 - reset) * encodeds[-1]
        
        all_actor_output = {k: torch.concat(v) for k, v in all_actor_output.items()}
        return all_actor_output, core_state  

In [17]:
def define_parser():

    parser = argparse.ArgumentParser(description="PyTorch Scalable Agent")

    parser.add_argument("--env", type=str, default="Sokoban-v0",
                        help="Gym environment.")
    parser.add_argument("--env_disable_noop", action="store_true",
                        help="Disable noop in environment or not. (sokoban only)")

    parser.add_argument("--xpid", default=None,
                        help="Experiment id (default: None).")

    # Training settings.
    parser.add_argument("--disable_checkpoint", action="store_true",
                        help="Disable saving checkpoint.")
    parser.add_argument("--savedir", default="~/RS/thinker/logs/torchbeast",
                        help="Root dir where experiment data will be saved.")
    parser.add_argument("--num_actors", default=48, type=int, metavar="N",
                        help="Number of actors (default: 48).")
    parser.add_argument("--total_steps", default=100000000, type=int, metavar="T",
                        help="Total environment steps to train for.")
    parser.add_argument("--batch_size", default=32, type=int, metavar="B",
                        help="Learner batch size.")
    parser.add_argument("--unroll_length", default=20, type=int, metavar="T",
                        help="The unroll length (time dimension).")
    parser.add_argument("--num_buffers", default=None, type=int,
                        metavar="N", help="Number of shared-memory buffers.")
    parser.add_argument("--num_learner_threads", "--num_threads", default=1, type=int,
                        metavar="N", help="Number learner threads.")
    parser.add_argument("--disable_cuda", action="store_true",
                        help="Disable CUDA.")

    # Architecture settings
    parser.add_argument("--tran_dim", default=64, type=int, metavar="N",
                        help="Size of transformer hidden dim.")
    parser.add_argument("--tran_mem_n", default=16, type=int, metavar="N",
                        help="Size of transformer memory.")
    parser.add_argument("--tran_layer_n", default=3, type=int, metavar="N",
                        help="Number of transformer layer.")
    parser.add_argument("--tran_t", default=1, type=int, metavar="T",
                        help="Number of recurrent step for transformer.")
    parser.add_argument("--tran_ff_n", default=256, type=int, metavar="N",
                        help="Size of transformer ff .")
    parser.add_argument("--tran_skip", action="store_true",
                        help="Whether to enable skip conn.")
    parser.add_argument("--tran_norm_first", action="store_true",
                        help="Whether to use norm first in transformer.")
    parser.add_argument("--tran_rpos", action="store_true",
                        help="Whether to use relative position in transformer.")
    parser.add_argument("--tran_lstm", action="store_true",
                        help="Whether to use LSTM-transformer.")
    parser.add_argument("--tran_lstm_no_attn", action="store_true",
                        help="Whether to disable attention in LSTM-transformer.")
    parser.add_argument("--tran_erasep", action="store_true",
                        help="Whether to erase past memories if not planning.")

    parser.add_argument("--rec_t", default=5, type=int, metavar="N",
                        help="Number of planning steps.")
    parser.add_argument("--ste", action="store_true",
                        help="Whether to use ste backprop.")
    parser.add_argument("--gb_ste", action="store_true",
                        help="Whether to use gb-ste backprop.")
    parser.add_argument("--gb_ste_temp_max", default=1, type=int, metavar="N",
                        help="Beginning temp. for gb-ste.")
    parser.add_argument("--gb_ste_temp_min", default=0.5, type=int, metavar="N",
                        help="Ending temp. for gb-ste.")    

    # Loss settings.
    parser.add_argument("--entropy_cost", default=0.01,
                        type=float, help="Entropy cost/multiplier.")
    parser.add_argument("--baseline_cost", default=0.5,
                        type=float, help="Baseline cost/multiplier.")
    parser.add_argument("--reg_cost", default=1,
                        type=float, help="Reg cost/multiplier.")
    parser.add_argument("--discounting", default=0.97,
                        type=float, help="Discounting factor.")
    parser.add_argument("--lamb", default=0.97,
                        type=float, help="Lambda when computing trace.")
    parser.add_argument("--reward_clipping", default=10, type=int, 
                        metavar="N", help="Reward clipping.")
    parser.add_argument("--trun_bs", action="store_true",
                        help="Whether to add baseline as reward when truncated.")

    # Optimizer settings.
    parser.add_argument("--learning_rate", default=0.0004,
                        type=float, metavar="LR", help="Learning rate.")
    parser.add_argument("--disable_adam", action="store_true",
                        help="Use Aadm optimizer or not.")
    parser.add_argument("--alpha", default=0.99, type=float,
                        help="RMSProp smoothing constant.")
    parser.add_argument("--momentum", default=0, type=float,
                        help="RMSProp momentum.")
    parser.add_argument("--epsilon", default=0.01, type=float,
                        help="RMSProp epsilon.")
    parser.add_argument("--grad_norm_clipping", default=0.0, type=float,
                        help="Global gradient norm clip.")
    # yapf: enable

    return parser

parser = define_parser()
flags = parser.parse_args([])        

flags.xpid = None
flags.env = "Sokoban-v0"
flags.num_actors = 32
flags.batch_size = 32
flags.unroll_length = 20
flags.learning_rate = 0.0004
flags.entropy_cost = 0.001
flags.lamb = 1.

flags.trun_bs = False
flags.total_steps = 100000000
flags.disable_adam = False

flags.tran_t = 1
flags.tran_mem_n = 16
flags.tran_layer_n = 3
flags.tran_lstm = True
flags.tran_lstm_no_attn = True
flags.tran_norm_first = False
flags.tran_ff_n = 256
flags.tran_skip = True
flags.tran_erasep = False
flags.tran_dim = 64
flags.tran_rpos = True
flags.rec_t = 5
flags.ste = True
flags.gb_ste = False
flags.gb_ste_temp_max = 1
flags.gb_ste_temp_min = 0.5

env = create_env(flags)
obs_shape, num_actions = env.observation_space.shape, env.action_space.n
model_learner = Model(flags, obs_shape, num_actions=num_actions)
model_actor = Model(flags, obs_shape, num_actions=num_actions)
checkpoint = torch.load("../models/model_1.tar")
model_learner.load_state_dict(checkpoint["model_state_dict"])  
model_actor.load_state_dict(checkpoint["model_state_dict"])  

logging.basicConfig(format='%(message)s', level=logging.DEBUG)

In [18]:
mp.set_sharing_strategy('file_system')

if flags.xpid is None:
    flags.xpid = "torchbeast-%s" % time.strftime("%Y%m%d-%H%M%S")
plogger = file_writer.FileWriter(
    xpid=flags.xpid, xp_args=flags.__dict__, rootdir=flags.savedir
)

flags.device = None
if not flags.disable_cuda and torch.cuda.is_available(): 
    logging.info("Using CUDA.")
    flags.device = torch.device("cuda")
else:
    logging.info("Not using CUDA.")
    flags.device = torch.device("cpu")

checkpointpath = os.path.expandvars(
    os.path.expanduser("%s/%s/%s" % (flags.savedir, flags.xpid, "model.tar"))
)

if flags.num_buffers is None:  # Set sensible default for num_buffers.
    flags.num_buffers = max(2 * flags.num_actors, flags.batch_size)
if flags.num_actors >= flags.num_buffers:
    raise ValueError("num_buffers should be larger than num_actors")
if flags.num_buffers < flags.batch_size:
    raise ValueError("num_buffers should be larger than batch_size")

T = flags.unroll_length
B = flags.batch_size

env = create_env(flags)

actor_net = Actor_Wrapper(flags, model_actor, actor=None, rec_t=flags.rec_t)
buffers = create_buffers(flags, env.observation_space.shape, env.action_space.n)

actor_net.share_memory()

# Add initial RNN state.
initial_agent_state_buffers = []
for _ in range(flags.num_buffers):
    state = actor_net.initial_state(batch_size=1)
    for t in state:
        t.share_memory_()
    initial_agent_state_buffers.append(state)

actor_processes = []
ctx = mp.get_context()
free_queue = ctx.SimpleQueue()
full_queue = ctx.SimpleQueue()

for i in range(flags.num_actors):
    actor = ctx.Process(target=act, args=(flags, i, free_queue, full_queue,
            actor_net, buffers, initial_agent_state_buffers,),)
    actor.start()
    actor_processes.append(actor)

learner_net = Actor_Wrapper(flags, model_learner, actor=None, 
                            rec_t=flags.rec_t).to(device=flags.device)

if not flags.disable_adam:
    print("Using Adam...")        
    optimizer = torch.optim.Adam(learner_net.actor.parameters(),lr=flags.learning_rate)
else:
    print("Using RMS Prop...")
    optimizer = torch.optim.RMSprop(
        learner_net.actor.parameters(),
        lr=flags.learning_rate,
        momentum=flags.momentum,
        eps=flags.epsilon,
        alpha=flags.alpha,)
print("All parameters: ")
for k, v in learner_net.named_parameters(): print(k, v.numel())    

def lr_lambda(epoch):
    return 1 - min(epoch * T * B, flags.total_steps) / flags.total_steps

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

logger = logging.getLogger("logfile")
stat_keys = ["mean_episode_return", "episode_returns", "total_loss",
    "pg_loss", "baseline_loss", "entropy_loss"]
logger.info("# Step\t%s", "\t".join(stat_keys))

step, stats, last_returns, tot_eps = 0, {}, deque(maxlen=400), 0

def batch_and_learn(i, lock=threading.Lock()):
    """Thread target for the learning process."""
    #nonlocal step, stats, last_returns, tot_eps
    global step, stats, last_returns, tot_eps
    timings = prof.Timings()
    while step < flags.total_steps:
        timings.reset()
        batch, agent_state = get_batch(flags, free_queue, full_queue, buffers,
            initial_agent_state_buffers, timings,)
        stats = learn(flags, actor_net, learner_net, batch, agent_state, optimizer, 
            scheduler)
        if flags.gb_ste:
            learner_net.actor.temp = torch.tensor(np.maximum(flags.gb_ste_temp_max * 
                    np.exp(-step/flags.total_steps), flags.gb_ste_temp_min).item(), 
                    dtype=torch.float32, device=learner_net.actor.temp.device)
        last_returns.extend(stats["episode_returns"])
        tot_eps = tot_eps + len(stats["episode_returns"])
        timings.time("learn")
        with lock:
            to_log = dict(step=step)
            to_log.update({k: stats[k] for k in stat_keys})
            to_log.update({"trail_mean_episode_return": np.average(last_returns) if len(last_returns) > 0 else 0.,
                           "episode": tot_eps})
            plogger.log(to_log)
            step += T * B

    if i == 0:
        logging.info("Batch and learn: %s", timings.summary())

for m in range(flags.num_buffers):
    free_queue.put(m)

threads = []
for i in range(flags.num_learner_threads):
    thread = threading.Thread(
        target=batch_and_learn, name="batch-and-learn-%d" % i, args=(i,)
    )
    thread.start()
    threads.append(thread)

def checkpoint():
    if flags.disable_checkpoint:
        return
    logging.info("Saving checkpoint to %s", checkpointpath)
    torch.save(
        {
            "model_state_dict": actor_net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "flags": vars(flags),
        },
        checkpointpath,
    )

timer = timeit.default_timer
try:
    last_checkpoint_time = timer()
    while step < flags.total_steps:
        start_step = step
        start_time = timer()
        time.sleep(5)

        if timer() - last_checkpoint_time > 10 * 60:  # Save every 10 min.
            checkpoint()
            last_checkpoint_time = timer()

        sps = (step - start_step) / (timer() - start_time)
        if stats.get("episode_returns", None):
            mean_return = (
                "Return per episode: %.1f. " % stats["mean_episode_return"]
            )
        else:
            mean_return = ""
        total_loss = stats.get("total_loss", float("inf"))

        print_str =  "Steps %i @ %.1f SPS. Eps %i. L400 Return %f. Loss %f" % (step, sps, tot_eps, 
            np.average(last_returns) if len(last_returns) > 0 else 0., total_loss)

        for s in ["pg_loss", "baseline_loss", "entropy_loss", "reg_loss"]:
            if s in stats:
                print_str += " %s %f" % (s, stats[s])

        logging.info(print_str)
except KeyboardInterrupt:
    for thread in threads:
        thread.join()        
    # Try joining actors then quit.
else:
    for thread in threads:
        thread.join()
    logging.info("Learning finished after %d steps.", step)
finally:
    for _ in range(flags.num_actors):
        free_queue.put(None)
    for actor in actor_processes:
        actor.join(timeout=1)

checkpoint()
plogger.close()

Creating log directory: /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958
[INFO:289857 file_writer:104 2022-10-30 20:09:58,382] Creating log directory: /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958
Symlinked log directory: /home/sc/RS/thinker/logs/torchbeast/latest
[INFO:289857 file_writer:117 2022-10-30 20:09:58,383] Symlinked log directory: /home/sc/RS/thinker/logs/torchbeast/latest
Saving arguments to /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958/meta.json
[INFO:289857 file_writer:129 2022-10-30 20:09:58,383] Saving arguments to /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958/meta.json
Saving messages to /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958/out.log
[INFO:289857 file_writer:137 2022-10-30 20:09:58,384] Saving messages to /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958/out.log
Saving logs data to /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958/logs.csv
[INFO:289857 fil

actor size:  286316


[INFO:291828 train:191 2022-10-30 20:09:58,690] Actor 0 started.
[INFO:289857 4259067192:82 2022-10-30 20:09:58,697] # Step	mean_episode_return	episode_returns	total_loss	pg_loss	baseline_loss	entropy_loss


actor size:  286316
Using Adam...
All parameters: 
model.frameEncoder.conv1.weight 4608
model.frameEncoder.conv1.bias 64
model.frameEncoder.res1.0.conv1.weight 36864
model.frameEncoder.res1.0.bn1.weight 64
model.frameEncoder.res1.0.bn1.bias 64
model.frameEncoder.res1.0.conv2.weight 36864
model.frameEncoder.res1.0.bn2.weight 64
model.frameEncoder.res1.0.bn2.bias 64
model.frameEncoder.conv2.weight 73728
model.frameEncoder.conv2.bias 128
model.frameEncoder.res2.0.conv1.weight 147456
model.frameEncoder.res2.0.bn1.weight 128
model.frameEncoder.res2.0.bn1.bias 128
model.frameEncoder.res2.0.conv2.weight 147456
model.frameEncoder.res2.0.bn2.weight 128
model.frameEncoder.res2.0.bn2.bias 128
model.frameEncoder.res3.0.conv1.weight 147456
model.frameEncoder.res3.0.bn1.weight 128
model.frameEncoder.res3.0.bn1.bias 128
model.frameEncoder.res3.0.conv2.weight 147456
model.frameEncoder.res3.0.bn2.weight 128
model.frameEncoder.res3.0.bn2.bias 128
model.dynamicModel.res.0.conv1.weight 159201
model.dynami

[INFO:289857 4259067192:168 2022-10-30 20:10:03,703] Steps 0 @ 0.0 SPS. Eps 0. L400 Return 0.000000. Loss inf
Updated log fields: ['_tick', '_time', 'step', 'mean_episode_return', 'episode_returns', 'total_loss', 'pg_loss', 'baseline_loss', 'entropy_loss', 'trail_mean_episode_return', 'episode']
[INFO:289857 file_writer:189 2022-10-30 20:10:07,359] Updated log fields: ['_tick', '_time', 'step', 'mean_episode_return', 'episode_returns', 'total_loss', 'pg_loss', 'baseline_loss', 'entropy_loss', 'trail_mean_episode_return', 'episode']
[INFO:289857 4259067192:168 2022-10-30 20:10:08,708] Steps 640 @ 127.9 SPS. Eps 5. L400 Return -0.999999. Loss -33.505402 pg_loss -42.673157 baseline_loss 9.591001 entropy_loss -1.013017 reg_loss 0.589770
[INFO:289857 4259067192:168 2022-10-30 20:10:13,714] Steps 640 @ 0.0 SPS. Eps 5. L400 Return -0.999999. Loss -33.505402 pg_loss -42.673157 baseline_loss 9.591001 entropy_loss -1.013017 reg_loss 0.589770
[INFO:289857 4259067192:168 2022-10-30 20:10:18,719] S

[INFO:289857 4259067192:168 2022-10-30 20:13:18,903] Steps 16000 @ 127.9 SPS. Eps 133. L400 Return -0.846616. Loss 10.360522 pg_loss 1.805578 baseline_loss 9.115225 entropy_loss -1.018613 reg_loss 0.458333
[INFO:289857 4259067192:168 2022-10-30 20:13:23,908] Steps 16640 @ 127.9 SPS. Eps 138. L400 Return -0.844927. Loss 100.785751 pg_loss 86.309479 baseline_loss 15.141750 entropy_loss -1.019825 reg_loss 0.354341
[INFO:289857 4259067192:168 2022-10-30 20:13:28,914] Steps 16640 @ 0.0 SPS. Eps 138. L400 Return -0.844927. Loss 100.785751 pg_loss 86.309479 baseline_loss 15.141750 entropy_loss -1.019825 reg_loss 0.354341
[INFO:289857 4259067192:168 2022-10-30 20:13:33,919] Steps 17280 @ 127.9 SPS. Eps 144. L400 Return -0.845833. Loss 57.553970 pg_loss 51.052383 baseline_loss 7.359958 entropy_loss -1.023383 reg_loss 0.165014
[INFO:289857 4259067192:168 2022-10-30 20:13:38,925] Steps 17920 @ 127.9 SPS. Eps 149. L400 Return -0.857717. Loss -5.287713 pg_loss -8.284613 baseline_loss 3.961298 entro

[INFO:289857 4259067192:168 2022-10-30 20:16:39,118] Steps 32640 @ 127.9 SPS. Eps 272. L400 Return -0.858088. Loss 64.069046 pg_loss 58.211369 baseline_loss 6.309720 entropy_loss -1.007984 reg_loss 0.555939
[INFO:289857 4259067192:168 2022-10-30 20:16:44,124] Steps 33280 @ 127.9 SPS. Eps 277. L400 Return -0.857039. Loss 52.003502 pg_loss 47.394684 baseline_loss 5.235051 entropy_loss -1.012238 reg_loss 0.386005
[INFO:289857 4259067192:168 2022-10-30 20:16:49,130] Steps 33280 @ 0.0 SPS. Eps 277. L400 Return -0.857039. Loss 52.003502 pg_loss 47.394684 baseline_loss 5.235051 entropy_loss -1.012238 reg_loss 0.386005
[INFO:289857 4259067192:168 2022-10-30 20:16:54,135] Steps 33920 @ 127.9 SPS. Eps 282. L400 Return -0.856028. Loss 54.116577 pg_loss 48.284386 baseline_loss 6.429394 entropy_loss -1.016277 reg_loss 0.419076
[INFO:289857 4259067192:168 2022-10-30 20:16:59,140] Steps 34560 @ 127.9 SPS. Eps 288. L400 Return -0.849305. Loss 47.060219 pg_loss 39.956230 baseline_loss 7.602670 entropy_

[INFO:289857 4259067192:129 2022-10-30 20:19:59,312] Saving checkpoint to /home/sc/RS/thinker/logs/torchbeast/torchbeast-20221030-200958/model.tar
[INFO:289857 4259067192:168 2022-10-30 20:19:59,323] Steps 49280 @ 127.6 SPS. Eps 410. L400 Return -0.859999. Loss -12.523324 pg_loss -13.647356 baseline_loss 1.952675 entropy_loss -1.017601 reg_loss 0.188958
[INFO:289857 4259067192:168 2022-10-30 20:20:04,329] Steps 49280 @ 0.0 SPS. Eps 410. L400 Return -0.859999. Loss -12.523324 pg_loss -13.647356 baseline_loss 1.952675 entropy_loss -1.017601 reg_loss 0.188958
[INFO:289857 4259067192:168 2022-10-30 20:20:09,334] Steps 49920 @ 127.9 SPS. Eps 416. L400 Return -0.859999. Loss 3.253895 pg_loss -0.860927 baseline_loss 5.045345 entropy_loss -1.021411 reg_loss 0.090888
[INFO:289857 4259067192:168 2022-10-30 20:20:14,340] Steps 50560 @ 127.9 SPS. Eps 421. L400 Return -0.824299. Loss 152.498505 pg_loss 102.970818 baseline_loss 50.416328 entropy_loss -1.020107 reg_loss 0.131465
[INFO:289857 42590671

[INFO:289857 4259067192:168 2022-10-30 20:23:14,520] Steps 65280 @ 127.9 SPS. Eps 544. L400 Return -0.754299. Loss 2.859816 pg_loss -3.571741 baseline_loss 7.306978 entropy_loss -1.019160 reg_loss 0.143739
[INFO:289857 4259067192:168 2022-10-30 20:23:19,523] Steps 65280 @ 0.0 SPS. Eps 544. L400 Return -0.754299. Loss 2.859816 pg_loss -3.571741 baseline_loss 7.306978 entropy_loss -1.019160 reg_loss 0.143739
[INFO:289857 4259067192:168 2022-10-30 20:23:24,527] Steps 65920 @ 127.9 SPS. Eps 549. L400 Return -0.749299. Loss 31.629723 pg_loss 21.296101 baseline_loss 11.223036 entropy_loss -1.020484 reg_loss 0.131072


KeyboardInterrupt: 

In [ ]:
step = flags.total_steps + 1
for thread in threads:
     thread.join()

In [ ]:
for _ in range(flags.num_actors):
    free_queue.put(None)
for actor in actor_processes:
    actor.join(timeout=1)

In [ ]:
torch.save({"model_state_dict": actor_net.state_dict(),},"base/actor_net_tmp_2.tar")

In [ ]:
# Debug trainned actor

class Actor_Wrapper(nn.Module):    
    def __init__(self, flags, model, actor=None, rec_t=5):
        
        super(Actor_Wrapper, self).__init__()   
        self.model = model
        self.num_actions = model.num_actions        
        self.rec_t = rec_t
        self.obs_shape = (3 * num_actions + 4 + self.rec_t, 1, 1)
        if actor is None:
            self.actor = Actor_net(obs_shape=self.obs_shape, num_actions=self.num_actions, flags=flags)
        else:
            self.actor = actor   
            
    def initial_state(self, batch_size):
        return self.actor.initial_state(batch_size)
            
    def forward(self, x, core_state=None):                      
        # x is env_output object with:
        # frame: T x B x C x H x W
        # last_action: T x B
        # reward: T x B
        
        tot_step, bsz, _, _, _ = x['frame'].shape
        device = x['frame'].device        
        self.model.train(False)
        
        for step in range(tot_step):
        
            state = x['frame'][step]        
            action = F.one_hot(x['last_action'][step], self.num_actions)
            reward = x['reward'][step]   
            done = x['done'][step]
            
            rs, vs, logits, encodeds = self.model(state, action.unsqueeze(0), one_hot=True)
            encoded_reset = encodeds[0].clone()
            encoded = encodeds[0]

            for t in range(self.rec_t):
                
                a = action if t == 0 else im_action  
                r = (reward if t == 0 else rs[-1]).unsqueeze(-1)
                v = vs[-1].unsqueeze(-1)
                logit = logits[-1]                    
                
                if t == 0:
                    r0 = r.clone()
                    v0 = v.clone()
                    logit0 = logit.clone()
                    #print("p0: \n", F.softmax(logit, dim=-1).cpu().detach().numpy())
                
                time = F.one_hot(torch.tensor([t], device=device).long(), self.rec_t).tile([bsz, 1])                        

                actor_input = torch.concat([a, r, v, logit, r0, v0, logit0, time], dim=-1)     
                actor_input = actor_input.unsqueeze(-1).unsqueeze(-1)

                actor_output, core_state = self.actor(actor_input, done=done, core_state=core_state)
                
                if t == self.rec_t - 1:
                    if step == 0:
                        all_actor_output = {k: [v.unsqueeze(0)] for k, v in actor_output.items()}
                    else:
                        for k, v in actor_output.items(): all_actor_output[k].append(v.unsqueeze(0))
                
                reset = actor_output["reset"]
                im_action = actor_output["im_action"]
                
                np.set_printoptions(suppress=True)    
                #print("im_action: \n", actor_output["im_action"].cpu().detach().numpy())
                #print("reset: ", reset.cpu().detach().numpy())

                if t < self.rec_t - 1:                
                    rs, vs, logits, encodeds = self.model.forward_encoded(encoded, im_action.unsqueeze(0), 
                                                                          one_hot=True)
                    reset = reset.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
                    encoded = reset * encoded_reset + (1 - reset) * encodeds[-1]
        
        all_actor_output = {k: torch.concat(v) for k, v in all_actor_output.items()}
        #ret_keys = ["policy_logits", "baseline", "action", "model_loss", "reg_loss"]
        #all_actor_output = {k: all_actor_output[k] for k in ret_keys}
        return all_actor_output, core_state    

    
bsz = 16
eps_n = 500
    
actor_net = Actor_Wrapper(flags, model_actor, actor=None, rec_t=flags.rec_t)
checkpoint = torch.load("base/actor_net_tmp.tar")
actor_net.load_state_dict(checkpoint["model_state_dict"])
actor_net.train(False)    
actor_net.to(flags.device)

In [ ]:
env = gym.vector.SyncVectorEnv([lambda: SokobanWrapper(gym.make("Sokoban-v0"), noop=True)] * bsz)
env = Vec_Environment(env, bsz)
core_state = actor_net.initial_state(bsz)
core_state = tuple(v.to(flags.device) for v in core_state)
returns = []
obs = env.initial()

while(len(returns) <= eps_n):
#for t in range(5):
    #print("t: %d" %t)    
    with torch.no_grad():
        cur_returns = obs['episode_return']    
        obs = {k:v.to(flags.device) for k, v in obs.items()}    
        actor_out, core_state = actor_net(obs, core_state)
        obs = env.step(actor_out["action"][0])
        #print("p_sel: \n", F.softmax(actor_out["policy_logits"], dim=-1).detach().numpy())
        #print("action: ", actor_out["action"][0])
        if torch.any(obs['done']):
            returns.extend(cur_returns[obs['done']].numpy())

returns = returns[:eps_n]            
print("Finish %d episode: avg. return: %.2f (+-%.2f) " % (len(returns),
      np.average(returns), np.std(returns) / np.sqrt(len(returns))))        

In [ ]:
# Change the model to another one

parser = define_parser()        
flags = parser.parse_args("--env Sokoban-v0".split())        
flags.device = torch.device("cuda")
env = create_env(flags)
obs_shape, num_actions = env.observation_space.shape, env.action_space.n

model = Model(flags, obs_shape, num_actions=num_actions).to(device=flags.device)
checkpoint = torch.load("base/model_r2.tar")
model.load_state_dict(checkpoint["model_state_dict"])  

model.train(False)
actor_net.model = model

In [ ]:
# Testing actor wrapper with a one-step greedy actor

class Actor_test(nn.Module):
    # A one-step greedy actor (for testing only)
    def __init__(self, obs_shape, num_actions, rec_t):
        super(Actor_test, self).__init__()   
        self.obs_shape = obs_shape
        self.num_actions = num_actions
        self.rec_t = rec_t
        self.rs = torch.zeros(num_actions,device=flags.device)
        self.vs = []
    
    def forward(self, x, done, core_state=None):
        x = x[:, :, 0, 0]        
        #print("Input: ", x[0])
        bsz = x.shape[0]
        time = x[0, -self.rec_t:]
        t = torch.argmax(time, dim=0)        
        
        if t == 0:
            self.rs = torch.zeros(rec_t, bsz, device=x.device)
            self.vs = torch.zeros(rec_t, bsz, device=x.device)
        
        self.rs[t] = x[:,self.num_actions]
        self.vs[t] = x[:,self.num_actions+1]  
        
        im_action = t if t < self.num_actions else torch.zeros(1, device=x.device).long()
        im_action = F.one_hot(im_action.unsqueeze(0), self.num_actions).tile([bsz, 1])
        reset = torch.ones(bsz, device=x.device)
        
        policy_logits = (self.rs[1:1+self.num_actions] + flags.discounting * self.vs[1:1+self.num_actions]).transpose(0, 1) 
        policy_probs = F.softmax(20 * policy_logits, dim=1)
        action = torch.multinomial(policy_probs, num_samples=1)
        
        ret = {"im_action": im_action, 
               "reset": reset, 
               "policy_logits": policy_logits, 
               "policy_probs": policy_probs, 
               "action": action,
               "baseline": torch.zeros_like(reset),
               "reg_loss": torch.zeros_like(reset)}
        
        return (ret, None)

model = model_actor    
model.train(False)    
    
rec_t = 6        
actor = Actor_test(obs_shape=(model.num_actions + 2 + rec_t), num_actions=model.num_actions, rec_t=rec_t)        
actor_wrapper = Actor_Wrapper(None, model, actor, rec_t = 6)
        
bsz = 2
env = gym.vector.SyncVectorEnv([lambda: SokobanWrapper(gym.make("Sokoban-v0"), noop=True)] * bsz)
env = Vec_Environment(env, bsz)
obs = env.initial()
state = obs['frame'][0].to(flags.device).clone()
action = torch.zeros(bsz).long().to(flags.device)

#obs = {k:v.to(flags.device).view((1, 10)+v.shape[2:]) for k, v in obs.items()}
obs = {k:v.to(flags.device) for k, v in obs.items()}
actor_output, _ = actor_wrapper(obs)
print(F.softmax(20 * actor_output['policy_logits'], dim=-1))
action, prob, q_ret = n_step_greedy_model(state, action, model, 1, encoded=None, temp=20.)
print(prob)

<font size="5">Misc.</font>

In [ ]:
batch = get_batch_m(flags, buffers)
print(torch.max(batch["reward"]), (torch.max(batch["reward"]) == batch["reward"]).nonzero())
print(batch["done"].nonzero())

In [ ]:
batch["reward"][:, 12]

In [ ]:
# DEBUG LOSS

#batch = get_batch_m(flags, buffers)

model.train(False)

rs, vs, logits, _ = model(batch['frame'][0], batch['action'])
logits = logits[:-1]

target_rewards = batch['reward'][1:]
target_logits = batch['policy_logits'][1:]

target_vs = []
target_v = model(batch['frame'][-1], batch['action'][[-1]])[1][0]    

for t in range(vs.shape[0]-1, 0, -1):
    new_target_v = batch['reward'][t] + flags.discounting * (target_v * (~batch['done'][t]).float() +
                       vs[t-1] * (batch['truncated_done'][t]).float())
    target_vs.append(new_target_v.unsqueeze(0))
    target_v = new_target_v
target_vs.reverse()
target_vs = torch.concat(target_vs, dim=0)

# if done on step j, r_{j}, v_{j-1}, a_{j-1} has the last valid loss 
# rs is stored in the form of r_{t+1}, ..., r_{t+k}
# vs is stored in the form of v_{t}, ..., v_{t+k-1}
# logits is stored in the form of a{t}, ..., a_{t+k-1}

done_masks = []
done = torch.zeros(vs.shape[1]).bool().to(batch['done'].device)
for t in range(vs.shape[0]):
    done = torch.logical_or(done, batch['done'][t])
    done_masks.append(done.unsqueeze(0))

done_masks = torch.concat(done_masks[:-1], dim=0)

# compute final loss
huberloss = torch.nn.HuberLoss(reduction='none', delta=1.0)    
rs_loss = torch.sum(huberloss(rs, target_rewards) * (~done_masks).float())
#rs_loss = torch.sum(((rs - target_rewards) ** 2) * (~r_logit_done_masks).float())
vs_loss = torch.sum(huberloss(vs[:-1], target_vs) * (~done_masks).float())
#vs_loss = torch.sum(((vs[:-1] - target_vs) ** 2) * (~v_done_masks).float())
logits_loss = compute_cross_entropy_loss(logits, target_logits, done_masks)

# debug
ind = 21

target_vs = []
target_v = vs[-1]
for t in range(vs.shape[0]-1, 0, -1):        
    new_target_v = batch['reward'][t] + flags.discounting * (target_v * (~batch['done'][t]).float() +
                       vs[t-1] * (batch['truncated_done'][t]).float())
    print(t, 
          "reward %2f" % batch['reward'][t,ind].item(), 
          "bootstrap %2f" % (target_v * (~batch['done'][t]).float())[ind].item(), 
          "truncated %2f" % (vs[t-1] * (batch['truncated_done'][t]).float())[ind].item(),
          "vs[t-1] %2f" % vs[t-1][ind].item(),
          "new_targ %2f" % new_target_v[ind].item())
    target_vs.append(new_target_v.unsqueeze(0))    
    target_v = new_target_v
target_vs.reverse()
target_vs = torch.concat(target_vs, dim=0)   
print("done", batch["done"][:, ind])
print("done_masks", done_masks[:, ind])
print("vs: ", vs[:, ind])
print("target_vs: ", target_vs[:, ind])
print("reward: ", rs[:, ind])
print("target_reward: ", target_rewards[:, ind])
print("logits: ", logits[:, ind])
print("target_logits: ", target_logits[:, ind])

In [ ]:
# alt. version of computing loss by treading terminal state as absorbing state (as in MuZero)

def compute_loss_m(model, batch):

    rs, vs, logits, _ = model(batch['frame'][0], batch['action'])
    logits = logits[:-1]

    target_logits = batch['policy_logits'][1:].clone()
    target_rewards = batch['reward'][1:].clone()

    done_masks = []
    done = torch.zeros(vs.shape[1]).bool().to(batch['done'].device)

    c_logits = target_logits[0]
    c_state = batch['frame'][0]
    for t in range(vs.shape[0]-1):
        if t > 0: done = torch.logical_or(done, batch['done'][t])
        c_logits = torch.where(done.unsqueeze(-1), c_logits, target_logits[t])
        target_logits[t] = c_logits
        c_state = torch.where(done.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1), c_state, batch['frame'][t])  
        done_masks.append(done.unsqueeze(0))
    done_masks = torch.concat(done_masks, dim=0)
    done = torch.logical_or(done, batch['done'][-1])
    c_state = torch.where(done.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1), c_state, batch['frame'][-1])
    target_rewards = target_rewards * (~done_masks).float()

    target_vs = []
    target_v = model(c_state, batch['action'][[-1]])[1][0].detach()
    
    for t in range(vs.shape[0]-1, 0, -1):
        new_target_v = batch['reward'][t] + flags.discounting * target_v
        target_vs.append(new_target_v.unsqueeze(0))
        target_v = new_target_v
    target_vs.reverse()
    target_vs = torch.concat(target_vs, dim=0)
    
    # compute final loss
    huberloss = torch.nn.HuberLoss(reduction='none', delta=1.0)    
    rs_loss = torch.sum(huberloss(rs, target_rewards.detach()))
    #rs_loss = torch.sum(((rs - target_rewards) ** 2) * (~r_logit_done_masks).float())
    vs_loss = torch.sum(huberloss(vs[:-1], target_vs.detach()))
    #vs_loss = torch.sum(((vs[:-1] - target_vs) ** 2) * (~v_done_masks).float())
    logits_loss = compute_cross_entropy_loss(logits, target_logits.detach(), None)
    
    return rs_loss, vs_loss, logits_loss